<img src="images/sutd.png" alt="drawing" style="width:300px;"/>

## <center>50.040 Natural Language Processing, Summer 2021<center>
    
<center>**Due 17 June 2021, 5pm** <center>
Mini Project

**Write your student ID and name**


### STUDENT ID: 1004365 

### Name: Lee Jet Xuen

### Students with whom you have discussed (if any):    
1. Tay Sze Chang 1004301
2. Brandon Chong Wah Jin 

# Introduction

Language models are very useful for a wide range of applications, e.g., speech recognition and machine translation. Consider a sentence consisting of words $x_1, x_2, …, x_m$, where $m$ is the length of the sentence, the goal of language modeling is to model the probability of the sentence, where $m \geq 1$, $x_i \in V $ and $V$ is the vocabulary of the corpus:
$$p(x_1, x_2, …, x_m)$$
In this project, we are going to explore both statistical language model and neural language model on the [Wikitext-2](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/) datasets. Download wikitext-2 word-level data and put it under the ``data`` folder.

## Statistical  Language Model

A simple way is to view words as independent random variables (i.e., zero-th order Markovian assumption). The joint probability can be written as:
$$p(x_1, x_2, …, x_m)=\prod_{i=1}^m p(x_i)$$
However, this model ignores the word order information, to account for which, under the first-order Markovian assumption, the joint probability can be written as:
$$p(x_0, x_1, x_2, …, x_{m})= \prod_{i=1}^{m}p(x_i \mid x_{i-1})$$
Under the second-order Markovian assumption, the joint probability can be written as:
$$p(x_{-1}, x_0, x_1, x_2, …, x_{m})= \prod_{i=1}^{m}p(x_i \mid x_{i-2}, x_{i-1})$$
Similar to what we did in HMM, we will assume that $x_{-1}=START, x_0=START, x_{m} = STOP$ in this definition, where $START, STOP$ are special symbols referring to the start and the end of a sentence.







### Parameter estimation

Let's use $count(u)$ to denote the number of times the unigram $u$ appears in the corpus, use $count(v, u)$ to denote the number of times the bigram $v, u$ appears in the corpus, and $count(w, v, u)$ the times the trigram $w, v, u$ appears in the corpus, $u \in V \cup STOP$ and $w, v \in V \cup START$.

And the parameters of the unigram, bigram and trigram models can be obtained using maximum likelihood estimation (MLE).

- In the unigram model, the parameters can be estimated as: $$p(u) = \frac {count(u)}{c}$$, where $c$ is the total number of words in the corpus.
- In the bigram model, the parameters can be estimated as:
$$p(u \mid v) = \frac{count(v, u)}{count(v)}$$
- In the trigram model, the parameters can be estimated as:
$$p(u \mid w, v) = \frac{count(w, v, u)}{count(w, v)}$$




### Smoothing the parameters
#### Add-k Smoothing
Note, it is likely that many parameters of bigram and trigram models will be 0 because the relevant bigrams and trigrams involved do not appear in the corpus. If you don't have a way to handle these 0 probabilities, all the sentences that include such bigrams or trigrams will have probabilities of 0.

We'll use a Add-k Smoothing method to fix this problem, the smoothed parameters can be estimated as:
\begin{equation}
p_{add-k}(u)= \frac{count(u)+k}{c+k|V^*|}
\end{equation}
\begin{equation}
p_{add-k}(u \mid v)= \frac{count(v, u)+k}{count(v)+k|V^*|}
\end{equation}
\begin{equation}
p_{add-k}(u \mid w, v)= \frac{count(w, v, u)+k}{count(w, v)+k|V^*|}
\end{equation}

where $k \in (0, 1)$ is the parameter of this approach, and $|V^*|$ is the size of the vocabulary $V^*$, here $V^*= V \cup STOP$. One way to choose the value of $k$ is by
optimizing the perplexity of the development set, namely to choose the value that minimizes the perplexity.
#### Interpolation
There is another way for smoothing which is named as **interpolation**. In interpolation, we always mix the probability estimates from
all the n-gram estimators, weighing and combining the trigram, bigram, and unigram counts. In simple linear interpolation, we combine different order n-grams by linearly interpolating all the models. Thus, we estimate the trigram probability $p(w_n|w_{n-2},w_{n-1})$ by mixing together the unigram, bigram, and trigram probabilities, each weighted by a $\lambda$:
\begin{align}
\hat{p}(w_n|w_{n-2},w_{n-1}) = \lambda_1p(w_n|w_{n-2},w_{n-1})+\lambda_2p(w_n|w_{n-1})+\lambda_3p(w_n)
\end{align}
such that the $\lambda$s sum to 1:
\begin{align}
\sum_i\lambda_i=1
\end{align}
In addition, $\lambda_1,\lambda_2,\lambda_3\geq 0$.

### Perplexity

Given a test set $D^{\prime}$ consisting of sentences $X^{(1)}, X^{(2)}, …, X^{(|D^{\prime}|)}$, each sentence $X^{(j)}$ consists of words $x_1^{(j)}, x_2^{(j)},…,x_{n_j}^{(j)}$, we can measure the probability of each sentence $X^{(j)}$, and the quality of the language model would be the probability it assigns to the entire set of test sentences, namely:
\begin{equation} 
\prod_{j=1}^{|D^{\prime}|}p(X^{(j)})
\end{equation}
Let's define average $log_2$ probability as:
\begin{equation} 
l=\frac{1}{c^{\prime}}\sum_{j=1}^{|D^{\prime}|}log_2p(X^{(j)})
\end{equation}
$c^{\prime}$ is the total number of words in the test set, $|D^{\prime}|$ is the number of sentences. And the perplexity is defined as:
\begin{equation} 
perplexity=2^{-l}
\end{equation}

The lower the perplexity, the better the language model.

In [ ]:
from collections import Counter, namedtuple
import itertools
import numpy as np
import sys

In [ ]:
with open("./data/wikitext-2/wiki.train.tokens", 'r', encoding='utf8') as f:
    text = f.readlines()
    train_sents = [line.lower().strip('\n').split() for line in text]
    train_sents = [s for s in train_sents if len(s)>0 and s[0] != '=']

In [ ]:
print(train_sents[1])

['the', 'game', 'began', 'development', 'in', '2010', ',', 'carrying', 'over', 'a', 'large', 'portion', 'of', 'the', 'work', 'done', 'on', 'valkyria', 'chronicles', 'ii', '.', 'while', 'it', 'retained', 'the', 'standard', 'features', 'of', 'the', 'series', ',', 'it', 'also', 'underwent', 'multiple', 'adjustments', ',', 'such', 'as', 'making', 'the', 'game', 'more', '<unk>', 'for', 'series', 'newcomers', '.', 'character', 'designer', '<unk>', 'honjou', 'and', 'composer', 'hitoshi', 'sakimoto', 'both', 'returned', 'from', 'previous', 'entries', ',', 'along', 'with', 'valkyria', 'chronicles', 'ii', 'director', 'takeshi', 'ozawa', '.', 'a', 'large', 'team', 'of', 'writers', 'handled', 'the', 'script', '.', 'the', 'game', "'s", 'opening', 'theme', 'was', 'sung', 'by', 'may', "'n", '.']


### Question 1 [code]
1. Implement the function **"compute_ngram"** that computes n-grams in the corpus.
 (Do not take the START and STOP symbols into consideration for now.) 
2. List 10 most frequent unigrams, bigrams and trigrams as well as their counts.(Hint: use the built-in function .most_common in Counter class)

In [ ]:
from nltk import ngrams

In [ ]:
def compute_ngram(sents, n):
    '''
    Compute n-grams that appear in "sents".
    param:
        sents: list[list[str]] --- list of list of word strings
        n: int --- "n" gram
    return:
        ngram_set: set{str} --- a set of n-grams (no duplicate elements)
        ngram_dict: dict{ngram: counts} --- a dictionary that maps each ngram to its number occurence in "sents";
        This dict contains the parameters of our ngram model. E.g. if n=2, ngram_dict={('a','b'):10, ('b','c'):13}
        
        You may need to use "Counter", "tuple" function here.
    '''
    ngram_set = None
    ngram_dict = None
    ### YOUR CODE HERE
    ngram_list = []
    
    for word in sents:
        for i in range(len(word)-n+1):
            ngram_list.append(tuple(word[i:i+n]))
            
    ngram_set = set(ngram_list)
    ngram_dict = Counter(ngram_list)

    ### END OF YOUR CODE
    return ngram_set, ngram_dict

In [ ]:
unigram_set, unigram_dict = compute_ngram(train_sents, 1)
print('unigram: %d' %(len(unigram_set)))
bigram_set, bigram_dict = compute_ngram(train_sents, 2)
print('bigram: %d' %(len(bigram_set)))
trigram_set, trigram_dict = compute_ngram(train_sents, 3)
print('trigram: %d' %(len(trigram_set)))

unigram: 28910
bigram: 577343
trigram: 1344047


In [ ]:
# List 10 most frequent unigrams, bigrams and trigrams as well as their counts.
### YOUR CODE HERE
print("-" * 117)
print("unigram:")
print(unigram_dict.most_common(10))
print("-" * 117)
print("bigram:")
print(bigram_dict.most_common(10))
print("-" * 117)
print("trigram:")
print(trigram_dict.most_common(10))
print("-" * 117)
### END OF YOUR CODE

---------------------------------------------------------------------------------------------------------------------
unigram:
[(('the',), 130519), ((',',), 99763), (('.',), 73388), (('of',), 56743), (('<unk>',), 53951), (('and',), 49940), (('in',), 44876), (('to',), 39462), (('a',), 36140), (('"',), 28285)]
---------------------------------------------------------------------------------------------------------------------
bigram:
[(('of', 'the'), 17242), (('in', 'the'), 11778), ((',', 'and'), 11643), (('.', 'the'), 11274), ((',', 'the'), 8024), (('<unk>', ','), 7698), (('to', 'the'), 6009), (('on', 'the'), 4495), (('the', '<unk>'), 4389), (('and', 'the'), 4331)]
---------------------------------------------------------------------------------------------------------------------
trigram:
[((',', 'and', 'the'), 1393), ((',', '<unk>', ','), 950), (('<unk>', ',', '<unk>'), 901), (('one', 'of', 'the'), 866), (('<unk>', ',', 'and'), 819), (('.', 'however', ','), 775), (('<unk>', '<unk>', '

### Question 2 [code]
In this part, we take the START and STOP symbols into consideration. So we need to pad the **train_sents** as described in "Statistical Language Model" before we apply "compute_ngram" function. For example, given a sentence "I like NLP", in a bigram model, we need to pad it as "START I like NLP STOP", in a trigram model, we need to pad it as "START START I like NLP STOP". For unigram model, it should be paded as "I like NLP STOP".

1. Implement the ``pad_sents`` function.
2. Pad ``train_sents``.
3. Apply ``compute_ngram`` function to these padded sents.
4. Implement ``ngram_prob`` function. Compute the probability for each n-gram in the variable **ngrams** according equations in **"Parameter estimation"**. List down the n-grams that have 0 probability. 



In [ ]:
###############################################
ngrams = list()
with open('data/ngram.txt','r') as f:
    for line in f:
        ngrams.append(line.strip('\n').split())
print(ngrams)
###############################################

[['the', 'computer'], ['go', 'to'], ['have', 'had'], ['and', 'the'], ['can', 'sea'], ['a', 'number', 'of'], ['with', 'respect', 'to'], ['in', 'terms', 'of'], ['not', 'good', 'bad'], ['first', 'start', 'with']]


In [ ]:
START = '<START>'
STOP = '<STOP>'
###################################
def pad_sents(sents, n):
    '''
    Pad the sents according to n.
    params:
        sents: list[list[str]] --- list of sentences.
        n: int --- specify the padding type, 1-gram, 2-gram, or 3-gram.
    return:
        padded_sents: list[list[str]] --- list of padded sentences.
    '''
    padded_sents = None
    ### YOUR CODE HERE
#     padded_sents = [[START]*(n-1)+s+[STOP] if n>1 else s for s in sents]
    padded_sents = []
    for word in sents:
        if n>1:
            padded_sents.append([START]*(n-1) + word + [STOP])
        else:
            padded_sents.append(word)
    ### END OF YOUR CODE
    return padded_sents

In [ ]:
uni_sents = pad_sents(train_sents, 1)
bi_sents = pad_sents(train_sents, 2)
tri_sents = pad_sents(train_sents, 3)

In [ ]:
unigram_set, unigram_dict = compute_ngram(uni_sents, 1)
bigram_set, bigram_dict = compute_ngram(bi_sents, 2)
trigram_set, trigram_dict = compute_ngram(tri_sents, 3)

In [ ]:
len(unigram_set),len(bigram_set),len(trigram_set)

(28910, 580825, 1363266)

In [ ]:
num_words = sum([v for _,v in unigram_dict.items()])
print(num_words)

2007146


In [ ]:
def ngram_prob(ngram, num_words, unigram_dic, bigram_dic, trigram_dic):
    '''
    params:
        ngram: list[str] --- a list that represents n-gram
        num_words: int --- total number of words
        unigram_dic: dict{ngram: counts} --- a dictionary that maps each 1-gram to its number of occurences in "sents";
        bigram_dic: dict{ngram: counts} --- a dictionary that maps each 2-gram to its number of occurence in "sents";
        trigram_dic: dict{ngram: counts} --- a dictionary that maps each 3-gram to its number occurence in "sents";
    return:
        prob: float --- probability of the "ngram"
    '''
    prob = None
    ### YOUR CODE HERE
    num = len(ngram)
    
    ngram = tuple(ngram)
    
    if num == 1:
        prob = unigram_dic[ngram] / num_words
    elif num == 2:
        prob = bigram_dic[ngram] / unigram_dic[ngram[:-1]]
    elif num == 3:
        prob = trigram_dic[ngram] / bigram_dic[ngram[:-1]]
    
    
    ### END OF YOUR CODE
    return prob

In [ ]:
ngram_prob(ngrams[0], num_words,unigram_dict, bigram_dict, trigram_dict)

9.960235674499498e-05

In [ ]:
### List down the n-grams that have 0 probability. 
### YOUR CODE HERE
for ngram in ngrams:
    if ngram_prob(ngram, num_words, unigram_dict, bigram_dict, trigram_dict) == 0:
        print(ngram)
### END OF YOUR CODE

['can', 'sea']
['not', 'good', 'bad']
['first', 'start', 'with']


### Question 3 [code]

1. Implement ``add_k_smoothing_ngram`` function to estimate ngram probability with ``add-k`` smoothing technique.
2. Implement ``interpolation_ngram`` function to estimate ngram probability with ``interpolation`` smoothing technique.
3. Implement ``perplexity`` function to compute the perplexity of the corpus "**valid_sents**" according to "**Perplexity**" section. The computation of $p(X^{(j)})$ depends on the n-gram model you choose.

In [ ]:
with open('data/wikitext-2/wiki.valid.tokens', 'r', encoding='utf8') as f:
    text = f.readlines()
    valid_sents = [line.lower().strip('\n').split() for line in text]
    valid_sents = [s for s in valid_sents if len(s)>0 and s[0] != '=']

uni_valid_sents = pad_sents(valid_sents, 1)
bi_valid_sents = pad_sents(valid_sents, 2)
tri_valid_sents = pad_sents(valid_sents, 3)

In [ ]:
def add_k_smoothing_ngram(ngram, k, num_words, unigram_dic, bigram_dic, trigram_dic):
    '''
    params:
        ngram: list[str] --- a list that represents n-gram
        k: float 
        num_words: int --- total number of words
        unigram_dic: dict{ngram: counts} --- a dictionary that maps each 1-gram to its number of occurences in "sents";
        bigram_dic: dict{ngram: counts} --- a dictionary that maps each 2-gram to its number of occurence in "sents";
        trigram_dic: dict{ngram: counts} --- a dictionary that maps each 3-gram to its number occurence in "sents";
    return:
        s_prob: float --- probability of the "ngram"
    '''
    s_prob = None
    V = len(unigram_dic)
    ### YOUR CODE HERE
    num = len(ngram)
    
    ngram = tuple(ngram)
    
    if num == 1:
        s_prob = (unigram_dic[ngram] + k) / (num_words + k*V)
    elif num == 2:
        s_prob = (bigram_dic[ngram] + k) / (unigram_dic[ngram[:-1]] + k*V)
    elif num == 3:
        s_prob = (trigram_dic[ngram] + k) / (bigram_dic[ngram[:-1]] + k*V)

    ### END OF YOUR CODE
    return s_prob

In [ ]:
def interpolation_ngram(ngram, lam, num_words, unigram_dic, bigram_dic, trigram_dic):
    '''
    params:
        ngram: list[str] --- a list that represents n-gram
        lam: list[float] --- a list of length 3.lam[0], lam[1] and lam[2] are correspondence to trigram, bigram and unigram,repectively.
                             If len(ngram) == 1, lam[0]=lam[1]=0, lam[2]=1. If len(ngram) == 2, lam[0]=0. lam[0]+lam[1]+lam[2] = 1.
        num_words: int --- total number of words
        unigram_dic: dict{ngram: counts} --- a dictionary that maps each 1-gram to its number of occurences in "sents";
        bigram_dic: dict{ngram: counts} --- a dictionary that maps each 2-gram to its number of occurence in "sents";
        trigram_dic: dict{ngram: counts} --- a dictionary that maps each 3-gram to its number occurence in "sents";
    return:
        s_prob: float --- probability of the "ngram"
    '''
    s_prob = None
    ### YOUR CODE HERE
    num = len(ngram)
    
    if num == 1:
        s_prob = ngram_prob(ngram, 
                            num_words,
                            unigram_dic,
                            bigram_dic,
                            trigram_dic)
    elif num == 2:
        x,y = ngram
        s_prob = lam[1]*ngram_prob(ngram,
                                   num_words,
                                   unigram_dic,
                                   bigram_dic,
                                   trigram_dic) +lam[2]*ngram_prob(y,
                                                                  num_words,
                                                                  unigram_dic,
                                                                  bigram_dic,
                                                                  trigram_dic)
    elif num == 3:
        x,y,z = ngram
        s_prob = lam[0]*ngram_prob(ngram,
                                   num_words,
                                   unigram_dic,
                                   bigram_dic,
                                   trigram_dic) + lam[1]*ngram_prob((y,z),
                                                                  num_words,
                                                                  unigram_dic,
                                                                  bigram_dic,
                                                                  trigram_dic) + lam[2]*ngram_prob(z,
                                                                                                  num_words,
                                                                                                  unigram_dic,
                                                                                                  bigram_dic,
                                                                                                  trigram_dic)
    
    ### END OF YOUR CODE
    return s_prob

In [ ]:
add_k_prob = add_k_smoothing_ngram(ngrams[5], 0.01, num_words, unigram_dict, bigram_dict, trigram_dict)
interpolation_prob = interpolation_ngram(ngrams[5], [0.6,0.3,0.1], num_words, unigram_dict, bigram_dict, trigram_dict)
print(ngrams[5])
print(add_k_prob, interpolation_prob)

['a', 'number', 'of']
0.5088478366553233 0.7254474091334492


In [ ]:
def perplexity(n, method, num_words, valid_sents, unigram_dic, bigram_dic, trigram_dic, k=0, lam=[0,0,1]):
    '''
    params:
        n: int --- n-gram model you choose 
        method: int ---- method == 0, use add_k_smoothing; method != 0, use interpolation method.
        num_words: int --- total number of words
        valid_sents: list[list[str]] --- list of sentences
        unigram_dic: dict{ngram: counts} --- a dictionary that maps each 1-gram to its number of occurences in "sents";
        bigram_dic: dict{ngram: counts} --- a dictionary that maps each 2-gram to its number of occurence in "sents";
        trigram_dic: dict{ngram: counts} --- a dictionary that maps each 3-gram to its number occurence in "sents";
        k: float --- The parameter of add_k_smoothing
        lam: list[float] --- a list of length 3. The parameter of interpolation. 
   return:
        ppl: float --- perplexity of valid_sents
    '''
    ppl = None
    ### YOUR CODE HERE
    loss = 0
    
    for s in valid_sents:
        for i in range(len(s)-n+1):
            if method == 0:
                loss += np.log(add_k_smoothing_ngram(s[i:i+n], k, num_words, unigram_dic, bigram_dic, trigram_dic))
            else:
                loss += interpolation_ngram(s[i:i+n], lam, num_words, unigram_dic, bigram_dic, trigram_dic)

    
    loss /= sum([len(o) for o in valid_sents])
    ppl = np.exp(-loss)
    
    ### END OF YOUR CODE
    return ppl

In [ ]:
perplexity(1, 0, num_words, uni_valid_sents, unigram_dict, bigram_dict, trigram_dict, k=0.1, lam=[0,0,1])

840.7346887991206

### Question 4 [code][written]
1. Based on add-k smoothing method, try out different $k\in [ 0.0001, 0.001, 0.01, 0.1, 0.5]$ and different n-gram model (unigram, bigram and trigram). Find the model and $k$ that gives the best perplexity on "**valid_sents**" (smaller is better).
2. Based on interpolation method, try out different $\lambda$ where $\lambda_1 = \lambda_2$ and $\lambda_3\in [0.1, 0.2, 0.4, 0.6, 0.8]$. Find the $\lambda$ that gives the best perplexity on "**valid_sents**" (smaller is better).
3. Based on the methods and parameters we provide, choose the method that peforms best on the validation data.

In [ ]:
n = [1,2,3]
k = [0.0001, 0.001, 0.01, 0.1, 0.5]
### YOUR CODE HERE (add-k smoothing method)

best_result = sys.maxsize
best_n = None
best_k = None
for kidx in k:
    for nidx in n:
        ppl  = perplexity(nidx, 0, num_words, valid_sents, unigram_dict, bigram_dict, trigram_dict, kidx)
        if ppl < best_result:
            best_result = ppl
            best_k = kidx
            best_n = nidx
print(f"The best n value is {best_n} and k value is {best_k} with a score of {best_result}.")

### END OF YOUR CODE

The best n value is 2 and k value is 0.01 with a score of 505.03474198080517.


In [ ]:
lambda_3 = [0.1, 0.2, 0.4, 0.6, 0.8]
### YOUR CODE HERE (interpolation method)

best_result = sys.maxsize
best_lam = None

for lam in lambda_3:
    lam_12 = (1 - lam)/2
    lamb_list = [lam_12, lam_12, lam]
    result  = perplexity(3, 1, num_words, valid_sents, unigram_dict, bigram_dict, trigram_dict, lam = lamb_list)
    if result < best_result:
        best_result = result
        best_lam = lamb_list

print(f"The best lambda value is {best_lam} with a score of {best_result}.")
### END OF YOUR CODE

ZeroDivisionError: division by zero

Based on the methods and parameters we provide, choose the method that peforms best on the validation data (**write your answer**): 

### Question 5 [code]

Evaluate the perplexity of the test data **test_sents** based on the best model you choose in **Question 4**.

In [ ]:
with open('data/wikitext-2/wiki.test.tokens', 'r', encoding='utf8') as f:
    text = f.readlines()
    test_sents = [line.lower().strip('\n').split() for line in text]
    test_sents = [s for s in test_sents if len(s)>0 and s[0] != '=']

uni_test_sents = pad_sents(test_sents, 1)
bi_test_sents = pad_sents(test_sents, 2)
tri_test_sents = pad_sents(test_sents, 3)

In [ ]:
### YOUR CODE HERE

### END OF YOUR CODE

## Neural Language Model


<img src="images/LM.png" alt="drawing" style="width:500px;"/>

We will create a LSTM language model as shown in figure and train it on the Wikitext-2 dataset. 
The data generators (train\_iter, valid\_iter, test\_iter) have been provided. 
The word embeddings together with the parameters in the LSTM model will be learned from scratch.

[Pytorch](https://pytorch.org/tutorials/) and [torchtext](https://torchtext.readthedocs.io/en/latest/index.html#) are required in this part. Do not make any changes to the provided code unless you are requested to do so. 

### Question 6 [code]
- Implement the ``__init__`` function in ``LangModel`` class. *Note: the code implementation should allow switching between unidirectional LSTM and bidirectional LSTM easily*
- Implement the ``forward`` function in ``LangModel`` class.
- Complete the training code in train function and the testing code in test function.
- Train two models - **Unidirectional LSTM** and **Bidirectional LSTM**. Compute the perplexity of the test data "test_iter" using the trained models. The test perplexity of both trained models should be below 150.

**Important Note: Make sure that "torchtext <= 0.11", as newer version might have torchtext.legacy removed**

In [1]:
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 19.2 MB/s 
     |████████████████████████████████| 831.4 MB 2.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.


In [31]:
import torchtext
import torch
import torch.nn.functional as F
from torchtext.legacy.datasets import WikiText2
from torch import nn, optim
from torchtext.legacy import data
from nltk import word_tokenize
import nltk
import numpy as np
nltk.download('punkt')
torch.manual_seed(222)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
def tokenizer(text):
    '''Tokenize a string to words'''
    return word_tokenize(text)

START = '<START>'
STOP = '<STOP>'
#Load and split data into three parts
TEXT = data.Field(lower=True, tokenize=tokenizer, init_token=START, eos_token=STOP)
train, valid, test = WikiText2.splits(TEXT) 

In [33]:
#Build a vocabulary from the train dataset
TEXT.build_vocab(train)
print('Vocabulary size:', len(TEXT.vocab))

Vocabulary size: 28907


In [34]:
BATCH_SIZE = 64
# the length of a text feeding to the RNN layer
BPTT_LEN = 32           
# train, validation, test data
train_iter, valid_iter, test_iter = data.BPTTIterator.splits((train, valid, test),
                                                                batch_size=BATCH_SIZE,
                                                                bptt_len=BPTT_LEN,
                                                                repeat=False)

In [35]:
#Generate a batch of train data
batch = next(iter(train_iter))
text, target = batch.text, batch.target
print('Size of text tensor',text.size())
print('Size of target tensor',target.size())

Size of text tensor torch.Size([32, 64])
Size of target tensor torch.Size([32, 64])


In [36]:
class LangModel(nn.Module):
    def __init__(self, lang_config):
        super(LangModel, self).__init__()
        self.vocab_size = lang_config['vocab_size']
        self.emb_size = lang_config['emb_size']
        self.hidden_size = lang_config['hidden_size']
        self.num_layer = lang_config['num_layer']
        
        self.embedding = None
        self.lstm = None
        self.linear = None
        
        ### TODO: 
        ###    1. Initialize 'self.embedding' with nn.Embedding function and 2 variables we have initialized for you
        ###    2. Initialize 'self.lstm' with nn.LSTM function and 4 variables we have initialized for you 
        ###    3. Initialize 'self.linear' with nn.Linear function and 2 variables we have initialized for you
        ### Reference:
        ###        https://pytorch.org/docs/stable/nn.html
        
        ### YOUR CODE HERE (3 lines)
        self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
        self.rnn = nn.LSTM(self.emb_size, self.hidden_size, self.num_layer, bidirectional=lang_config['bidirectional'])
        self.linear = nn.Linear(self.hidden_size, self.vocab_size)

        ### END OF YOUR CODE
        
    def forward(self, batch_sents, hidden=None):
        '''
        params:
            batch_sents: torch.LongTensor of shape (sequence_len, batch_size)
        return:
            normalized_score: torch.FloatTensor of shape (sequence_len, batch_size, vocab_size)
        '''
        normalized_score = None
        hidden = hidden
        ### TODO:
        ###      1. Feed the batch_sents to self.embedding  
        ###      2. Feed the embeddings to self.lstm. Remember to pass "hidden" into self.lstm, even if it is None. But we will 
        ###         use "hidden" when implementing greedy search.
        ###      3. Apply linear transformation to the output of self.lstm
        ###      4. Apply 'F.log_softmax' to the output of linear transformation
        ###
        ### YOUR CODE HERE (4 lines)
        
        x = self.embedding(batch_sents)
        out, hidden = self.rnn(x, hidden)
        score = self.linear(out)
        normalized_score = F.log_softmax(score, dim=-1)
        
        ### END OF YOUR CODE
        return normalized_score, hidden

In [37]:
def train(model, train_iter, valid_iter, vocab_size, criterion, optimizer, num_epochs):
    for n in range(num_epochs):
        train_loss = 0
        target_num = 0
        model.train()
        for batch in train_iter:
            
            text, targets = batch.text.to(device), batch.target.to(device)
            loss = None
            
            ### we don't consider "hidden" here. So according to the default setting, "hidden" will be None
            ### YOU CODE HERE (~5 lines)
            
            optimizer.zero_grad()
            pred, hidden = model(text)
            loss = criterion(pred.view(-1, vocab_size), targets.view(-1))
            loss.backward()
            optimizer.step()

            ### END OF YOUR CODE
            ##########################################
            train_loss += loss.item() * targets.size(0) * targets.size(1)
            target_num += targets.size(0) * targets.size(1)

        train_loss /= target_num

        # monitor the loss of all the predictions
        val_loss = 0
        target_num = 0
        model.eval()
        for batch in valid_iter:
            text, targets = batch.text.to(device), batch.target.to(device)
            
            prediction,_ = model(text)
            loss = criterion(prediction.view(-1, vocab_size), targets.view(-1))
            
            val_loss += loss.item() * targets.size(0) * targets.size(1)
            target_num += targets.size(0) * targets.size(1)
        val_loss /= target_num

        print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(n+1, train_loss, val_loss))   

In [38]:
def test(model, vocab_size, criterion, test_iter):
    '''
    params: 
        model: LSTM model
        test_iter: test data
    return:
        ppl: perplexity 
    '''
    ppl = None
    test_loss = 0
    target_num = 0
    with torch.no_grad():
        for batch in test_iter:
            text, targets = batch.text.to(device), batch.target.to(device)

            prediction,_ = model(text)
            loss = criterion(prediction.view(-1, vocab_size), targets.view(-1))

            test_loss += loss.item() * targets.size(0) * targets.size(1)
            target_num += targets.size(0) * targets.size(1)

        test_loss /= target_num
        
        ### Compute perplexity according to "test_loss"
        ### Hint: Consider how the loss is computed.
        ### YOUR CODE HERE(1 line)
        
        ppl = np.exp(test_loss)
        
        ### END OF YOUR CODE
        return ppl

In [21]:
config = {
    'vocab_size':vocab_size,
    'emb_size':128,
    'hidden_size':128,
    'num_layer':1,
    'bidirectional': False
}

LM = LangModel(config)
LM = LM.to(device)

In [22]:
num_epochs=10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = len(TEXT.vocab)

criterion = nn.NLLLoss(reduction='mean')
optimizer = optim.Adam(LM.parameters(), lr=1e-3, betas=(0.7, 0.99))

In [23]:
train(LM, train_iter, valid_iter, vocab_size, criterion, optimizer, num_epochs)

Epoch: 1, Training Loss: 6.0626, Validation Loss: 5.1756
Epoch: 2, Training Loss: 5.3989, Validation Loss: 4.9562
Epoch: 3, Training Loss: 5.1284, Validation Loss: 4.8596
Epoch: 4, Training Loss: 4.9565, Validation Loss: 4.8106
Epoch: 5, Training Loss: 4.8304, Validation Loss: 4.7828
Epoch: 6, Training Loss: 4.7316, Validation Loss: 4.7654
Epoch: 7, Training Loss: 4.6503, Validation Loss: 4.7539
Epoch: 8, Training Loss: 4.5810, Validation Loss: 4.7466
Epoch: 9, Training Loss: 4.5204, Validation Loss: 4.7430
Epoch: 10, Training Loss: 4.4667, Validation Loss: 4.7423


In [28]:
test(LM, vocab_size, criterion, test_iter)

99.49475529850511

In [39]:
config = {
    'vocab_size':vocab_size,
    'emb_size':128,
    'hidden_size':128,
    'num_layer':1,
    'bidirectional': True
}

biLSTM = LangModel(config)
biLSTM = biLSTM.to(device)

In [40]:
train(biLSTM, train_iter, valid_iter, vocab_size, criterion, optimizer, num_epochs)

RuntimeError: ignored

In [ ]:
test(biLSTM, vocab_size, criterion, test_iter)

### Question 7 [code][written]
<img src="images/greedy.png" alt="drawing" style="width:500px;"/>

When we use trained language model to generate a sentence given a start token, we can choose ``greedy search``.

As shown above, ``greedy search`` algorithm will pick the token which has the highest probability and feed it to the language model as input in the next time step. The model will generate ``max_len`` number of tokens at most.

- Implement ``word_greedy_search``

In [ ]:
def word_greedy_search(model, start_token, max_len):
    '''
    param:
        model: nn.Module --- language model
        start_token: str --- e.g. 'he'
        max_len: int --- max number of tokens generated
    return:
        strings: list[str] --- list of tokens, e.g., ['he', 'was', 'a', 'member', 'of',...]
    '''
    model.eval()
    ID = TEXT.vocab.stoi[start_token]
    strings = [start_token]
    hidden = None
    
    ### You may find TEXT.vocab.itos useful.
    ### YOUR CODE HERE
    
    predicts = torch.ones(1, 1).long().to(device) * ID
    for idx in range(max_len):
        
        logits, hidden = model(predicts, hidden)
        predicts = torch.argmax(logits[-1,:,:], dim=-1)
        strings.append(TEXT.vocab.itos[predicts.cpu().numpy()[0]])
        
        if strings[-1] == '<eos>': 
            break
        
        predicts.unsqueeze_(0)

    ### END OF YOUR CODE 
    print(strings)    

In [ ]:
word_greedy_search(LM, 'he', 64)

#### Review Question: Based on your understanding, can we use the **Bidirectional LSTM** for this language generation (decoding) task? Explain why?

**write your explanation:**

### Question 8 [code][written]
- We will use the hidden vectors (the working memory) of LSTM as the contextual embeddings. Implement ``contextual_embedding`` function.
- Use the ``contextual_embedding`` function to get the contextual embeddings of the word "sink" in four sequences "wood does not sink in water", "a small water leak will sink the ship", "there are plates in the kitchen sink" and "the kitchen sink was full of dirty dishes". Then calculate the cosine similarity of "sink" from each pair of sequences. Assume that $\boldsymbol{w}_1$ and $\boldsymbol{w}_2$ are embeddings of "sink" in sequences "wood does not sink in water" and "a small water leak will sink the ship" respectively. The cosine similarity can be calculated as 

\begin{align}
similarity = cos(\theta) = \frac{\boldsymbol{w}^{\rm T}_1\boldsymbol{w}_2}{||\boldsymbol{w}_1||_2||\boldsymbol{w}_2||_2}
\end{align}
Give the explanation of the results. 

In [ ]:
def contextual_embedding(model, sentence):
    '''
    params: 
        model: nn.Module --- language model
        sentence -- list[str]: list of tokens, e.g., ['I', 'am',...]
    return:
        embeddings -- numpy array of shape (length of sentence, word embedding size)
    '''
    model.eval()
    hidden = None
    
    ### YOUR CODE HERE 
    
    
    
    ### END OF YOUR CODE
    return embeddings

In [ ]:
sink_seq1 = "wood does not sink in water"
sink_seq2 = "a small water leak will sink the ship"
sink_seq3 = "there are plates in the kitchen sink"
sink_seq4 = "the kitchen sink was full of dirty dishes"

### YOUR CODE HERE 

### END OF YOUR CODE


***write your explanation:***

#### Review Question: Based on your understanding, can we use the **Bidirectional LSTM** for this contextual embedding task? Explain why?

**write your explanation:**

### Requirements:
- This is an individual report.
- Complete the code using Python.
- List students with whom you have discussed if there are any.
- Follow the honor code strictly.

### Free GPU Resources
We suggest that you run neural language models on machines with GPU(s). Google provides the free online platform [Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb), a research tool for machine learning education and research. It’s a Jupyter notebook environment that requires no setup to use as common packages have been  pre-installed. Google users can have access to a Tesla T4 GPU (approximately 15G memory). Note that when you connect to a GPU-based VM runtime, you are given a maximum of 12 hours at a time on the VM.

It is convenient to upload local Jupyter Notebook files and data to Colab, please refer to the [tutorial](https://colab.research.google.com/notebooks/io.ipynb). 

In addition, Microsoft also provides the online platform [Azure Notebooks](https://notebooks.azure.com/help/introduction) for research of data science and machine learning, there are free trials for new users with credits.